In [44]:
import pandas as pd
import re
from imblearn.over_sampling import ADASYN, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [45]:
# Import data
train = pd.read_csv("../data/train_complete.csv")
test = pd.read_csv("../data/test_complete.csv")

## Resampling

In [46]:
# Dataprep - toda transformação deve ser criada sobre o teste e somente aplicada sobre o treino
def apply_random_oversampling(df, oversample_col='', exclude_category=None):
    '''
    df: dataframe with all columns
    oversample_col: column to apply the oversampling
    exclude_category: list of categories on oversample_col to set apart while oversampling
    '''
    df_cat = pd.DataFrame(columns = df.columns)
    df_ = df.copy()
    
    # Separando categorias específica se necessário
    if exclude_category is not None:
        
        for cat in exclude_category:
            
            df_cat = pd.concat([df_cat, df[df[oversample_col]==cat]]).reset_index(drop=True)
            
            # removendo categorias que não serão oversampled
            df_ = df_[df[oversample_col]!=cat].reset_index()
    
    # Separando coluna de oversampling
    X = df_.drop(columns=[oversample_col])
    y = df_[oversample_col]

    # Aplicando Random Oversampling
    oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = oversampler.fit_resample(X, y)

    df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
    df_resampled[oversample_col] = y_resampled

    # Exibir o DataFrame balanceado
    print(df_resampled[oversample_col].value_counts())
    
    # inserindo de volta as categorias que foram separadas
    if not df_cat.empty:
        df_resampled = pd.concat([df_resampled, df_cat]).reset_index(drop=True)
        
    return df_resampled

In [47]:
def apply_random_undersampling(df, undersample_col='', strategy={}):
    '''
    df: dataframe with all columns
    undersample_col: column to apply the undersampling
    srtategy: dictionary with the category as key and number of samples requested as value
    '''

    X = df.drop(columns=[undersample_col])
    y = df[undersample_col]

    # aplicando undersampling
    undersampler = RandomUnderSampler(sampling_strategy=strategy, random_state=42)
    X_resampled, y_resampled = undersampler.fit_resample(X, y)

    df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
    df_resampled[undersample_col] = y_resampled

    # Exibir o DataFrame balanceado e a distribuição de classes
    print(df_resampled[undersample_col].value_counts())
    
    return df_resampled

In [48]:
train.columns

Index(['P0', 'P1_a', 'P1_a_1', 'P1_b', 'P1_c', 'P1_d', 'P1_e', 'P1_e_1',
       'P1_e_2', 'P1_e_3',
       ...
       'P6_g_Postgres/MySQL(OneHot)', 'P6_g_Presto(OneHot)',
       'P6_g_Snowflake(OneHot)', 'P6_g_Teradata(OneHot)', 'P6_g_nan(OneHot)',
       'P1_l(OrdEnc)', 'P2_e(OrdEnc)', 'P2_g(OrdEnc)', 'P2_i(OrdEnc)',
       'P2_j(OrdEnc)'],
      dtype='object', length=739)

In [49]:
col_raca = 'P1_c'
col_genero = 'P1_b'
col_idade ='P1_a_1'
# col_regiao = 'P1_i_2'

cols_resampling = [col_raca, col_genero, col_idade ] # , col_regiao]

df_resampled = train.copy()

for col in cols_resampling:
    df_resampled = apply_random_oversampling(df_resampled, oversample_col=col)

/var/folders/bk/g3qj80rj34g417z03l7_tsjc0000gn/T/ipykernel_88466/4237983600.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_resampled[oversample_col] = y_resampled


Branca                  2732
Parda                   2732
Preta                   2732
Prefiro não informar    2732
Amarela                 2732
Outra                   2732
Indígena                2732
Name: P1_c, dtype: int64


/var/folders/bk/g3qj80rj34g417z03l7_tsjc0000gn/T/ipykernel_88466/4237983600.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_resampled[oversample_col] = y_resampled


Masculino               13650
Feminino                13650
Outro                   13650
Prefiro não informar    13650
Name: P1_b, dtype: int64
25-29    17297
30-34    17297
40-44    17297
45-49    17297
35-39    17297
22-24    17297
17-21    17297
50-54    17297
55+      17297
Name: P1_a_1, dtype: int64


/var/folders/bk/g3qj80rj34g417z03l7_tsjc0000gn/T/ipykernel_88466/4237983600.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_resampled[oversample_col] = y_resampled


In [50]:
print(df_resampled.shape)
print(df_resampled.info())

(155673, 739)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155673 entries, 0 to 155672
Columns: 739 entries, P0 to P1_a_1
dtypes: bool(650), int64(6), object(83)
memory usage: 202.2+ MB
None


In [51]:
# !pip install seaborn

In [52]:
onehot = ["P1_m", "P3_c","P4_a","P4_b", "P4_c", "P4_d", "P4_e", "P4_g", "P4_j", "P5_b", "P6_a", "P6_b", "P6_g", "P6_h", "P7_a", "P7_b", "P7_d", "P8_a", "P8_b", "P8_c", "P8_d", "P2_b", "P2_d", "P2_f", "P2_o", "P2_r", "P6_c", "P6_d", "P6_e", "P6_f"]
ordered = ["P1_l", "P2_e", "P2_g", "P2_i", "P2_j"]

# df_feat_selected = df_resampled[ [x +endswith("(OneHot)") for x in onehot ] + [x +endswith("(OrdEnc)") for x in ordered] ]


In [53]:
import re

# Define the patterns for filtering the columns
onehot_pattern = f"^({'|'.join(onehot)}).*\\(OneHot\\)$"
ordered_pattern = f"^({'|'.join(ordered)}).*\\(OrdEnc\\)$"

# Apply the filtering to select the matching columns
df_feat_selected = df_resampled[
    df_resampled.columns[df_resampled.columns.str.contains(onehot_pattern) | 
                         df_resampled.columns.str.contains(ordered_pattern)]
]

/var/folders/bk/g3qj80rj34g417z03l7_tsjc0000gn/T/ipykernel_88466/3852787019.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_resampled.columns[df_resampled.columns.str.contains(onehot_pattern) |
/var/folders/bk/g3qj80rj34g417z03l7_tsjc0000gn/T/ipykernel_88466/3852787019.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_resampled.columns.str.contains(ordered_pattern)]


In [54]:
df_feat_selected.head()

,P1_m_Ciências Biológicas/ Farmácia/ Medicina/ Área da Saúde(OneHot),P1_m_Ciências Sociais(OneHot),P1_m_Computação / Engenharia de Software / Sistemas de Informação/ TI(OneHot),P1_m_Economia/ Administração / Contabilidade / Finanças/ Negócios(OneHot),P1_m_Estatística/ Matemática / Matemática Computacional/ Ciências Atuariais(OneHot),P1_m_Marketing / Publicidade / Comunicação / Jornalismo(OneHot),P1_m_Outra opção(OneHot),P1_m_Outras Engenharias(OneHot),P1_m_Química / Física(OneHot),P1_m_nan(OneHot),...,P6_g_Postgres/MySQL(OneHot),P6_g_Presto(OneHot),P6_g_Snowflake(OneHot),P6_g_Teradata(OneHot),P6_g_nan(OneHot),P1_l(OrdEnc),P2_e(OrdEnc),P2_g(OrdEnc),P2_i(OrdEnc),P2_j(OrdEnc)
0,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,3,-1,1,2,3
1,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,True,6,-1,1,7,6
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,True,3,-1,2,1,5
3,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,True,3,-1,0,2,3
4,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,3,-1,1,3,5


In [ ]:
#